In [1]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import warnings  # 警告处理

#from sklearn.linear_model.coordinate_descent import ConvergenceWarning  # 警告处理
from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV  # 回归模型
from sklearn.tree import DecisionTreeRegressor  # 回归决策树模型
from sklearn.model_selection import train_test_split  # 数据分割
from sklearn.preprocessing import MinMaxScaler  # 数据归一化 (分类)
from sklearn.preprocessing import StandardScaler  # 数据归一化(回归)

from sklearn import tree
from sklearn.decomposition import PCA  # 主成分分析
from sklearn.pipeline import Pipeline  # 管道
from sklearn.model_selection import GridSearchCV  # 网格搜索交叉验证，用于选择最优的参数

from sklearn.preprocessing import label_binarize  # 数据二值化
from sklearn.decomposition import PCA  # 降维
from sklearn.ensemble import RandomForestClassifier  # 分类随机森林模型
from sklearn.ensemble import RandomForestRegressor  # 回归随机森林模型
from sklearn import metrics  # ROC/AUC

In [12]:
# 1.数据引入
train=pd.read_csv('lntrain5.csv')
test=pd.read_csv('lntest5.csv')
#所有列都进入决策
train=train[['category','lnfans', 'lnlikes', 'lnviewers_y','date', 'count', 'lnviewers_x', 'sum', 'customer_unit_price', 'lnsales','lnsales_value']]
test=test[['category','lnfans', 'lnlikes', 'lnviewers_y','date', 'count', 'lnviewers_x', 'sum', 'customer_unit_price', 'lnsales','lnsales_value']]
# 2.数据处理


train[np.isinf(train)]=np.nan
train=train.dropna(axis=0,how='any')
train = train.values
print(train)


test[np.isinf(test)]=np.nan
test=test.dropna(axis=0,how='any')
test = test.values
print(test)

# 3.数据分割
X_train, Y_train = np.split(train, (10,), axis=1)
X_test,Y_test=np.split(test, (10,), axis=1)
print ("训练数据集样本数目：%d, 测试数据集样本数目：%d" % (X_train.shape[0], X_test.shape[0]))

# 4.数据归一化(回归)
from sklearn.preprocessing import StandardScaler  # 数据归一化(回归)
ss = StandardScaler()  # 创建归一化模型
X_train = ss.fit_transform(X_train, Y_train)  # 训练模型并转换数据
X_test = ss.transform(X_test)  # 转换数据

[[   2.           12.81503355   11.44021068 ...   84.
     9.42003388   13.85089146]
 [   2.           12.86519344   11.10448243 ... 1644.58
     8.14583961   15.55108144]
 [   2.           12.62278098   11.98470373 ...   58.12
     6.68085468   10.74338308]
 ...
 [   3.           14.13962862    9.47566038 ...  209.89
     6.57228254   11.91887979]
 [   2.           14.38847909   11.61066451 ... 1770.77
     7.78239034   15.26156281]
 [   2.           12.56578697   10.81089225 ...   48.15
     7.74109909   11.61548115]]
[[  6.          12.51189052  10.45937736 ... 124.01         6.56807791
   11.38847387]
 [  6.          14.05452746   8.11822195 ... 132.2          6.25575004
   11.1400529 ]
 [  7.          11.74184431  11.80817756 ...  57.17         6.59304453
   10.63903308]
 ...
 [  2.          18.1643007   15.56390537 ... 106.71        13.80664033
   18.47679458]
 [  2.          12.92337083  11.59509514 ... 128.08         7.08590146
   11.93852256]
 [  2.          12.98611515   9.80

In [18]:

## b.线性回归模型
# 1、构建模型
lr = LinearRegression() # 构建线性回归模型
lr.fit(X_train, Y_train)  # 模型训练
lr_y_hat = lr.predict(X_test)  # 模型预测
print(lr.coef_)
lr_score = lr.score(X_test, Y_test)  # 模型评估
print("线性回归正确率：%.2f%%" % (lr_score * 100))
# 2、计算MAPE值
errors = abs(lr_y_hat - Y_test) # 计算误差
mape = 100 * (errors / Y_test) # mean absolute percentage error (MAPE)
where_are_inf = np.isinf(mape)
mape[where_are_inf] = 0
print(np.mean(mape))
# 3、计算NMSE值
a = (lr_y_hat - Y_test)**2 # 分子
y=np.mean(Y_test)
b=(lr_y_hat-y)**2 # 分母
nmse=np.mean(a)/np.mean(b)
print(nmse)

[[-3.59323432e-01  8.69456587e-02 -1.75787793e-03 -5.24151802e-02
  -1.80625118e-04 -9.49398500e-02  3.86638562e-01 -2.35188984e-02
   7.12442475e-01  1.28289393e+00]]
线性回归正确率：57.25%
10.925536299838114
0.7362410984071883


D:\Anaconda_exe\lib\site-packages\ipykernel_launcher.py:12: RuntimeWarning: divide by zero encountered in true_divide
  if sys.path[0] == '':


In [ ]:
print("线性回归正确率：%.2f%%" % (lr_score * 100))
## c.Lasso回归模型
ls = LassoCV(alphas=np.logspace(-3,1,20)) # 构建LASSO模型
ls.fit(X_train, Y_train)  # 模型训练
ls_y_hat = ls.predict(X_test)  # 模型预测
ls_score = ls.score(X_test, Y_test)  # 模型评估
print("Lasso回归正确率：%.2f%%" % (ls_score * 100))
## d.Ridge回归模型
rg = RidgeCV(alphas=np.logspace(-3,1,20)) # 构建LASSO模型
rg.fit(X_train, Y_train)  # 模型训练
rg_y_hat = rg.predict(X_test)  # 模型预测
rg_score = rg.score(X_test, Y_test)  # 模型评估
print("Ridge回归正确率：%.2f%%" % (rg_score * 100))